In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import plot_model
from sklearn.impute import SimpleImputer


df = pd.read_csv('../csv/merged_data.csv')

numerical_features = ['temp','visibility','dew_point','feels_like','pressure','humidity','wind_speed','wind_deg','wind_gust','rain_1h','snow_1h','clouds_all','weather_id', 'year','month','day']
X = df[numerical_features]
y = df['aqi']

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


/var/folders/6m/1wyk0cv146d91z1nsb6cxh6w0000gn/T/ipykernel_25459/2663223116.py:11: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../csv/merged_data.csv')


In [31]:
def build_model(hp):
    inputs = Input(shape=(X_train.shape[1],))
    x = inputs
    # Define a tunable number of layers
    for i in range(hp.Int('num_layers', 2, 20)):
        x = Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                  activation='relu')(x)
    outputs = Dense(1, activation='linear')(x)  # Change to 'sigmoid' or 'softmax' for classification
    model = Model(inputs, outputs)
    
    model.compile(optimizer='adam',
                  loss='mean_squared_error',  # Change to 'binary_crossentropy' or 'categorical_crossentropy' for classification
                  metrics=['mean_squared_error'])
    return model

In [33]:
import kerastuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective='val_mean_squared_error',  # Change accordingly if you have a different metric
    max_trials=10,  # Adjust the number of trials or add `executions_per_trial`
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='aqi_prediction'
)

# Start search
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, mse = best_model.evaluate(X_test, y_test)
print(f"Test MSE: {mse}")


Trial 4 Complete [00h 27m 44s]
val_mean_squared_error: 139.97947692871094

Best val_mean_squared_error So Far: 136.27496337890625
Total elapsed time: 01h 35m 48s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
3                 |17                |num_layers
288               |448               |units_0
96                |512               |units_1
96                |32                |units_2
96                |32                |units_3
64                |32                |units_4
320               |32                |units_5
512               |32                |units_6
32                |32                |units_7
32                |32                |units_8
32                |32                |units_9
256               |32                |units_10
64                |32                |units_11
128               |32                |units_12
128               |32                |units_13
352               |32                |units_14
448           

KeyboardInterrupt: 

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize the Neural Network
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Adding the second hidden layer
model.add(Dense(units=32, activation='relu'))

# Adding the output layer
model.add(Dense(units=1, activation='linear'))  # 'linear' activation because this is a regression problem

# Compiling the Neural Network
model.compile(optimizer ='adam', loss='mean_squared_error')


In [ ]:
# Training the Neural Network
model.fit(X_train, y_train, batch_size=32, epochs=10)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import plot_model
from kerastuner.tuners import RandomSearch

# Assuming `df` is your DataFrame and `target` is the name of your target column
X = df.drop('target', axis=1)
y = df['target']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numeric and categorical columns
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

# Scale numeric features
scaler = StandardScaler()
X_train_numeric = scaler.fit_transform(X_train[numeric_cols])
X_test_numeric = scaler.transform(X_test[numeric_cols])

# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False)
X_train_categorical = encoder.fit_transform(X_train[categorical_cols])
X_test_categorical = encoder.transform(X_test[categorical_cols])

# Concatenate numeric and categorical features
X_train_preprocessed = np.concatenate([X_train_numeric, X_train_categorical], axis=1)
X_test_preprocessed = np.concatenate([X_test_numeric, X_test_categorical], axis=1)


In [26]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt

# Predicting the Test set results
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
rmse = sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


6430/6430 [==============================] - 6s 986us/step
Mean Squared Error: 150.3888049505987
Mean Absolute Error: 7.938101958563345
Root Mean Squared Error: 12.263311337098097
R-squared: 0.3904708644959457
